## 15.6 循環参照は、メモリをリークすることもある

### 循環参照させる

In [2]:
use std::rc::Rc;
use std::cell::RefCell;
use List::{Cons, Nil};

#[derive(Debug)]
enum List {
    Cons(i32, RefCell<Rc<List>>),
    Nil,
}

impl List {
    fn tail(&self) -> Option<&RefCell<Rc<List>>> {
        match *self {
            Cons(_, ref item) => Some(item),
            Nil => None,
        }
    }
}


In [4]:
let a = Rc::new(Cons(5, RefCell::new(Rc::new(Nil))));

println!("a initial rc count = {}", Rc::strong_count(&a));
println!("a next item = {:?}", a.tail());

a initial rc count = 1
a next item = Some(RefCell { value: Nil })


`[a] => [5 | ] => Nil`

In [5]:
let b = Rc::new(Cons(10, RefCell::new(Rc::clone(&a))));

println!("a rc count after b creation = {}", Rc::strong_count(&a));
println!("b initial rc count = {}", Rc::strong_count(&b));
println!("b next item = {:?}", b.tail());

a rc count after b creation = 2
b initial rc count = 1
b next item = Some(RefCell { value: Cons(5, RefCell { value: Nil }) })


`[b] => [10 | ] => [a] => [5 | ] => Nil`

In [6]:
if let Some(link) = a.tail() {
  *link.borrow_mut() = Rc::clone(&b);
}

println!("b rc count after changing a = {}", Rc::strong_count(&b));
println!("a rc count after changing a = {}", Rc::strong_count(&a));

b rc count after changing a = 2
a rc count after changing a = 2


`[b] => [10 | ] => [a] => [5 | ] => [b] => [10 | ] => [a] => ...`

In [7]:
// This will overflow the stack
// println!("a next item = {:?}", a.tail());

### 循環参照を回避する: Rc\<T\>をWeak\<T\>に変換する

#### 木データ構造を作る: 子ノードのあるNode

In [8]:
#[derive(Debug)]
struct Node {
  value: i32,
  childredn: RefCell<Vec<Rc<Node>>>
}

In [10]:
let leaf = Rc::new(Node {
  value: 3,
  childredn: RefCell::new(vec![]),
});
leaf

Node { value: 3, childredn: RefCell { value: [] } }

In [11]:
let branch = Rc::new(Node {
  value: 5,
  childredn: RefCell::new(vec![Rc::clone(&leaf)])
});
branch

Node { value: 5, childredn: RefCell { value: [Node { value: 3, childredn: RefCell { value: [] } }] } }

branch -> leaf は辿れる

branch <- leaf は辿れない状態


#### 子供から親に参照を追加する

実現すべきこと
- Node構造体定義にparentフィールドを追加する
- 循環参照はしない

参照関係
- 親ノードは小ノードを所有する
- 子ノードは親ノードを所有しない

In [14]:
use std::rc::Weak;

#[derive(Debug)]
struct Node {
  value: i32,
  parent: RefCell<Weak<Node>>,
  childredn: RefCell<Vec<Rc<Node>>>,
}

In [18]:
let leaf = Rc::new(Node {
  value: 3,
  parent: RefCell::new(Weak::new()),
  childredn: RefCell::new(vec![]),
});
println!("leaf parent = {:?}", leaf.parent.borrow().upgrade());
leaf

leaf parent = None


Node { value: 3, parent: RefCell { value: (Weak) }, childredn: RefCell { value: [] } }

In [19]:
let branch = Rc::new(Node {
  value: 5,
  parent: RefCell::new(Weak::new()),
  childredn: RefCell::new(vec![Rc::clone(&leaf)]),
});
branch

Node { value: 5, parent: RefCell { value: (Weak) }, childredn: RefCell { value: [Node { value: 3, parent: RefCell { value: (Weak) }, childredn: RefCell { value: [] } }] } }

In [20]:
*leaf.parent.borrow_mut() = Rc::downgrade(&branch);


In [24]:
println!("leaf parent = {:?}", leaf.parent.borrow().upgrade());


leaf parent = Some(Node { value: 5, parent: RefCell { value: (Weak) }, childredn: RefCell { value: [Node { value: 3, parent: RefCell { value: (Weak) }, childredn: RefCell { value: [] } }] } })


In [23]:
println!("leaf parent = {:?}", leaf.parent.borrow());

leaf parent = (Weak)


In [25]:
let leaf = Rc::new(Node {
  value: 3,
  parent: RefCell::new(Weak::new()),
  childredn: RefCell::new(vec![]),
});

println!(
  "leaf strong = {}, weak = {}",
  Rc::strong_count(&leaf),
  Rc::weak_count(&leaf),
);

leaf strong = 1, weak = 0


In [26]:
{
  let branch = Rc::new(Node {
    value: 5,
    parent: RefCell::new(Weak::new()),
    childredn: RefCell::new(vec![Rc::clone(&leaf)]),
  });

  *leaf.parent.borrow_mut() = Rc::downgrade(&branch);

  println!(
    "branch strong = {}, weak = {}",
    Rc::strong_count(&branch),
    Rc::weak_count(&branch),
  );

  println!(
    "leaf strong = {}, weak = {}",
    Rc::strong_count(&leaf),
    Rc::weak_count(&leaf),
  );
}

branch strong = 1, weak = 1
leaf strong = 2, weak = 0


()

`branch` has been dropped due to scope even `branch` was referenced from `leaf`. As long as the reference is weak, Rust will drop the variable.

In [27]:
println!("leaf parent = {:?}", leaf.parent.borrow().upgrade());

leaf parent = None


In [28]:
println!(
  "leaf strong = {}, weak = {}",
  Rc::strong_count(&leaf),
  Rc::weak_count(&leaf),
);

leaf strong = 1, weak = 0
